<img src="data/pic7.png">


****
## Step 1: Sentence segmentation

전체 텍스트 (문단들 전체)를 문장으로 나누는 것이다. 보통 온점을 기준으로 문장을 나눈다. 그러나  문서가 깨끗하지 않으면 다른 복잡한 기술이 필요하기도 한다.

****
## Step2: Word Tokenization

문장을 얻었으니, 단어word 단위로 문장을 쪼갤 수 있다. 이를 tokenization이라고 하고 쪼개진 단위는 word 또는 token이라고도 한다. 공백이 있는 영어 같은 언어에서 쉬운 편이다. 온점을 token의 분리 단위에 포함시키기도 한다. 

****
## Step3: Predicting Parts of Speech for Each Token

<img src="data/pic1.png">

<img src="data/pic2.png">

token이 noun, verb, adjective 등인지 구분하는 것이 필요하다. 각 word의 역할을 아는것은 문장이 뭘 말하고 있는지 아는데 도움을 준다. 

****
## Step4: Text Lemmatization

<img src="data/pic3.png">

I had a pony.

I had two ponies

두 문장 모두 noun pony에 대해 이야기하고 있으나 inflection이 다르다. 두 단어를 다른 단어로 인식하는 것은 곤란하다. lemmatization은 단어의 lemma(basic form)을 알아내는 것이다. 

****
## Step5: Stop Words

<img src="data/pic4.png">

stopword를 filter out하고 통계 분석을 해야 한다. 왜냐하면 너무 자주 나와서 노이즈를 발생시키기 때문이다. 

****
## Step6: Dependancy Parsing

<img src="data/pic5.png">


모든 단어가 자신의 문장에서 서로 어떻게 연관되는지를 알아내는 것을 지칭한다. 
목적은 하나의 parent word에서 뻗어나간 트리를 각 단어와 연결시키는 것이다. 
거기서 더 나아가, 두 단어 사이의 관계의 타입을 예측할 수도 있다. 

****
## Step 6b: Finding Noun Phrases

여태까지는 모든 단어를 각각 독립된 entity로 취급해왔다. 그러나 가끔 그룹핑해서 하나의 생각을 나타내는 것으로 간주하는 것이 더 말이 된다. 이걸해야하는지 아닌지는 우리의 목표에 달렸다. 

****
## Step 7: Named Entity Recognition (NER)

<img src="data/pic6.png">

NER의 목적은 단어가 나타내는 real-world 콘셉트에 따라 단어를 detect하고 레이블링하는 것이다. 예를 들어

- 사람 이름
- 회사 이름
- 지리 위치
- 물건 이름
- 날짜 시간
- 돈의 액수
- 특별한 사건

****
## Step 8: Conference Resolution

<img src="data/pic6.png">

여기까지는 문장 하나를 파악했다. 그런데 영어에는 대명사가 있다. conference resolution은 대명사가 지칭하는 것을 파악하는 것이다. 

In [3]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = """London is the capital and most populous city of England and 
the United Kingdom.  Standing on the River Thames in the south east 
of the island of Great Britain, London has been a major settlement 
for two millennia. It was founded by the Romans, who named it Londinium.
"""

# Parse the text with spaCy. This runs the entire pipeline.
doc = nlp(text)

# 'doc' now contains a parsed version of text. We can use it to do anything we want!
# For example, this will print out all the named entities that were detected:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

London (GPE)
England (GPE)
the United Kingdom (GPE)
the River Thames (LOC)
Great Britain (GPE)
London (GPE)
two millennia (DATE)
Romans (NORP)
Londinium (LOC)


In [4]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# Replace a token with "REDACTED" if it is a name
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.ent_type_ == "PERSON":
        return "[REDACTED] "
    else:
        return token.string

# Loop through all the entities in a document and check if they are names
def scrub(text):
    doc = nlp(text)
    for ent in doc.ents:
        ent.merge()
    tokens = map(replace_name_with_placeholder, doc)
    return "".join(tokens)

s = """
In 1950, Alan Turing published his famous article "Computing Machinery and Intelligence". In 1957, Noam Chomsky’s 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.
"""

print(scrub(s))


In 1950, [REDACTED] published his famous article "Computing Machinery and Intelligence". In 1957, [REDACTED] 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.



In [1]:
import spacy
import textacy.extract

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = """London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
"""

# Parse the document with spaCy
doc = nlp(text)

# Extract semi-structured statements
statements = textacy.extract.semistructured_statements(doc, "London")

# Print the results
print("Here are the things I know about London:")

for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

Here are the things I know about London:
 - the capital and most populous city of England and  the United Kingdom.  

 - a major settlement  for two millennia.  
